In [1]:
import pandas as pd
import numpy as np
import sys
import os
import bs4
from IPython.display import display_html,clear_output, HTML
import re
from datetime import datetime
import ast
import itertools
from tqdm import tqdm,trange
from file_tools import *
from request_tools import *
from parse_tools import *
from load_tools import *
from model_tools import *
import pyarrow as pa

pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 30)
pd.set_option('compute.use_numexpr', False)

import warnings
from pandas.errors import PerformanceWarning
from sklearn.exceptions import DataConversionWarning, ConvergenceWarning
pd.options.mode.use_inf_as_na = True
warnings.filterwarnings('ignore', category=PerformanceWarning)
warnings.filterwarnings('ignore', category=DataConversionWarning)
warnings.filterwarnings('ignore', category=ConvergenceWarning)

import pandas as pd
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif, RFE, mutual_info_regression
from sklearn.linear_model import LogisticRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from scipy.stats import pearsonr
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVR


In [2]:
# Load predictions results
preds = pd.read_csv('./08-experiments/dataset-ss0110-mnmx-7fg/WL/20231206-174521/predictions.csv',header=[0,1,2,3,4],index_col=[0,1,2])
preds

f_classif   
                                                          percentile   
                                                                   5   
                                                              y_true   
                                                  Unnamed: 3_level_4   
                       index Team_id                                   
/leagues/NBA_2015.html 8     /teams/BOS/2015.html                  0  \
                             /teams/BRK/2015.html                  0   
                             /teams/CHI/2015.html                  1   
                             /teams/CHO/2015.html                  1   
                             /teams/CLE/2015.html                  0   
                             /teams/DEN/2015.html                  0   
                             /teams/DET/2015.html                  1   
                             /teams/GSW/2015.html                  1   
                             /teams/HOU/2015.html                  1   
                             /teams/LAC/2015.html                  0   
                             /teams/MEM/2015.html                  1   
                             /teams/MIL/2015.html                  0   
                             /teams/MIN/2015.html                  0   
                             /teams/NOP/2015.html                  0   
                             /teams/NYK/2015.html                  0   
...                                                              ...   
/leagues/NBA_2023.html 81    /teams/MIA/2023.html                  1   
                             /teams/MIL/2023.html                  0   
                             /teams/MIN/2023.html                  1   
                             /teams/NOP/2023.html                  0   
                             /teams/NYK/2023.html                  0   
                             /teams/OKC/2023.html                  1   
                             /teams/ORL/2023.html                  0   
                             /teams/PHI/2023.html                  1   
                             /teams/PHO/2023.html                  0   
                             /teams/POR/2023.html                  0   
                             /teams/SAC/2023.html                  0   
                             /teams/SAS/2023.html                  1   
                             /teams/TOR/2023.html                  1   
                             /teams/UTA/2023.html                  0   
                             /teams/WAS/2023.html                  0   

                                                                             
                                                                             
                                                                             
                                                  AdaBoostClassifier         
                                                                   0  1  2   
                       index Team_id                                         
/leagues/NBA_2015.html 8     /teams/BOS/2015.html                  1  1  1  \
                             /teams/BRK/2015.html                  0  0  0   
                             /teams/CHI/2015.html                  0  0  0   
                             /teams/CHO/2015.html                  0  0  1   
                             /teams/CLE/2015.html                  1  1  1   
                             /teams/DEN/2015.html                  1  1  1   
                             /teams/DET/2015.html                  0  0  0   
                             /teams/GSW/2015.html                  1  1  1   
                             /teams/HOU/2015.html                  1  1  1   
                             /teams/LAC/2015.html                  0  0  0   
                             /teams/MEM/2015.html                  1  1  1   
                             /teams/MIL/2015.html                  1

Get the validate set predictions only

In [3]:
def __process_predictions_validation_set__(preds):
    y_preds = preds.copy().T.drop_duplicates(keep='first').T
    y_true = y_preds[y_preds.columns[y_preds.columns.get_level_values(-2).isin(['y_true'])]]
    is_Val = y_preds[y_preds.columns[y_preds.columns.get_level_values(-2).isin(['is_Val'])]]
    y_preds = y_preds.drop(columns=['y_true','is_Val'],level=-2)

    y_preds_val = pd.DataFrame(index=y_preds.index,columns=y_preds.columns.droplevel(-1).unique())
    for column_indexer in y_preds_val.columns.unique():
        y_preds_val[column_indexer] = y_preds[column_indexer].mul(is_Val.values,axis=0).sum(axis=1)
    y_preds_val.insert(0,'y_true',y_true)
    return y_preds_val

y_preds_val = __process_predictions_validation_set__(preds)
y_preds_val

y_true          f_classif   
                                                                 percentile   
                                                                          5   
                                                         AdaBoostClassifier   
                       index Team_id                                          
/leagues/NBA_2015.html 8     /teams/BOS/2015.html      0                  1  \
                             /teams/BRK/2015.html      0                  0   
                             /teams/CHI/2015.html      1                  0   
                             /teams/CHO/2015.html      1                  1   
                             /teams/CLE/2015.html      0                  1   
                             /teams/DEN/2015.html      0                  1   
                             /teams/DET/2015.html      1                  0   
                             /teams/GSW/2015.html      1                  1   
                             /teams/HOU/2015.html      1                  1   
                             /teams/LAC/2015.html      0                  0   
                             /teams/MEM/2015.html      1                  1   
                             /teams/MIL/2015.html      0                  1   
                             /teams/MIN/2015.html      0                  0   
                             /teams/NOP/2015.html      0                  0   
                             /teams/NYK/2015.html      0                  0   
...                                                  ...                ...   
/leagues/NBA_2023.html 81    /teams/MIA/2023.html      1                  1   
                             /teams/MIL/2023.html      0                  1   
                             /teams/MIN/2023.html      1                  0   
                             /teams/NOP/2023.html      0                  1   
                             /teams/NYK/2023.html      0                  1   
                             /teams/OKC/2023.html      1                  1   
                             /teams/ORL/2023.html      0                  0   
                             /teams/PHI/2023.html      1                  1   
                             /teams/PHO/2023.html      0                  1   
                             /teams/POR/2023.html      0                  0   
                             /teams/SAC/2023.html      0                  0   
                             /teams/SAS/2023.html      1                  0   
                             /teams/TOR/2023.html      1                  0   
                             /teams/UTA/2023.html      0                  0   
                             /teams/WAS/2023.html      0                  1   

                                                                         
                                                                         
                                                                         
                                                  KNeighborsClassifier   
                       index Team_id                                     
/leagues/NBA_2015.html 8     /teams/BOS/2015.html                    0  \
                             /teams/BRK/2015.html                    0   
                             /teams/CHI/2015.html                    1   
                             /teams/CHO/2015.html                    0   
                             /teams/CLE/2015.html                    0   
                             /teams/DEN/2015.html                    0   
                             /teams/DET/2015.html                    0   
                             /teams/GSW/2015.html                    0   
                             /teams/HOU/2015.html                    1   
                             /teams/LAC/2015.html                    0   
                             /teams/MEM/2015.html                    1   
                     

In [4]:
ModelStack.score(y_preds_val,score_names='classification')

,accuracy,f1,precision,recall
"(f_classif, percentile, 10, AdaBoostClassifier)",0.648487,0.646668,0.650032,0.643340
"(f_classif, percentile, 10, KNeighborsClassifier)",0.598835,0.597916,0.599290,0.596548
"(f_classif, percentile, 10, LogisticRegression)",0.650879,0.651114,0.650675,0.651555
"(f_classif, percentile, 10, MLPClassifier)",0.619892,0.623164,0.617845,0.628574
"(f_classif, percentile, 10, RandomForestClassifier)",0.636893,0.633001,0.639860,0.626287
"(f_classif, percentile, 10, RidgeClassifier)",0.651139,0.651356,0.650950,0.651763
"(f_classif, percentile, 10, SVC)",0.649579,0.649688,0.649486,0.649891
"(f_classif, percentile, 5, AdaBoostClassifier)",0.631850,0.632251,0.631563,0.632942
"(f_classif, percentile, 5, KNeighborsClassifier)",0.584382,0.584468,0.584347,0.584590
"(f_classif, percentile, 5, LogisticRegression)",0.639025,0.639118,0.638952,0.639285


In [6]:
X_d1 = y_preds_val.drop(columns='y_true')
y = y_preds_val['y_true']

# model_stack = ModelStackCV(models='classification', model_params='classification')
# predictions = model_stack.fit_predict(X, y, cv =3)
# scores = model_stack.score(predictions, score_names='classification')

y_preds_d1 = pd.read_csv('./08-experiments/dataset-ss0110-mnmx-7fg/WL/20231206-174521/deriv_1/predictions.csv',index_col=[0,1,2],header=[0,1])
y_preds_d1

y_true   
                                                  Unnamed: 3_level_1   
                       index Team_id                                   
/leagues/NBA_2015.html 8     /teams/BOS/2015.html                  0  \
                             /teams/BRK/2015.html                  0   
                             /teams/CHI/2015.html                  1   
                             /teams/CHO/2015.html                  1   
                             /teams/CLE/2015.html                  0   
                             /teams/DEN/2015.html                  0   
                             /teams/DET/2015.html                  1   
                             /teams/GSW/2015.html                  1   
                             /teams/HOU/2015.html                  1   
                             /teams/LAC/2015.html                  0   
                             /teams/MEM/2015.html                  1   
                             /teams/MIL/2015.html                  0   
                             /teams/MIN/2015.html                  0   
                             /teams/NOP/2015.html                  0   
                             /teams/NYK/2015.html                  0   
...                                                              ...   
/leagues/NBA_2023.html 81    /teams/MIA/2023.html                  1   
                             /teams/MIL/2023.html                  0   
                             /teams/MIN/2023.html                  1   
                             /teams/NOP/2023.html                  0   
                             /teams/NYK/2023.html                  0   
                             /teams/OKC/2023.html                  1   
                             /teams/ORL/2023.html                  0   
                             /teams/PHI/2023.html                  1   
                             /teams/PHO/2023.html                  0   
                             /teams/POR/2023.html                  0   
                             /teams/SAC/2023.html                  0   
                             /teams/SAS/2023.html                  1   
                             /teams/TOR/2023.html                  1   
                             /teams/UTA/2023.html                  0   
                             /teams/WAS/2023.html                  0   

                                                  AdaBoostClassifier         
                                                                   0  1  2   
                       index Team_id                                         
/leagues/NBA_2015.html 8     /teams/BOS/2015.html                  1  1  1  \
                             /teams/BRK/2015.html                  0  0  0   
                             /teams/CHI/2015.html                  0  0  0   
                             /teams/CHO/2015.html                  0  0  0   
                             /teams/CLE/2015.html                  1  1  1   
                             /teams/DEN/2015.html                  0  0  0   
                             /teams/DET/2015.html                  0  0  0   
                             /teams/GSW/2015.html                  1  1  1   
                             /teams/HOU/2015.html                  1  1  1   
                             /teams/LAC/2015.html                  0  0  0   
                             /teams/MEM/2015.html                  1  1  1   
                             /teams/MIL/2015.html                  0  0  0   
                             /teams/MIN/2015.html                  0  0  0   
                             /teams/NOP/2015.html                  0  0  0   
                             /teams/NYK/2015.html                  1  1  1   
...                                                              ... .. ..   
/leagues/NBA_2023.html 81    /teams/MIA/2023.html                  1  1  1   
                             /teams/MIL/2023.html    

In [7]:
y_preds_d1_val= __process_predictions_validation_set__(y_preds_d1)
y_preds_d1_val

y_true  AdaBoostClassifier   
                       index Team_id                                            
/leagues/NBA_2015.html 8     /teams/BOS/2015.html       0                   1  \
                             /teams/BRK/2015.html       0                   0   
                             /teams/CHI/2015.html       1                   0   
                             /teams/CHO/2015.html       1                   0   
                             /teams/CLE/2015.html       0                   1   
                             /teams/DEN/2015.html       0                   0   
                             /teams/DET/2015.html       1                   0   
                             /teams/GSW/2015.html       1                   1   
                             /teams/HOU/2015.html       1                   1   
                             /teams/LAC/2015.html       0                   0   
                             /teams/MEM/2015.html       1                   1   
                             /teams/MIL/2015.html       0                   0   
                             /teams/MIN/2015.html       0                   0   
                             /teams/NOP/2015.html       0                   0   
                             /teams/NYK/2015.html       0                   1   
...                                                   ...                 ...   
/leagues/NBA_2023.html 81    /teams/MIA/2023.html       1                   1   
                             /teams/MIL/2023.html       0                   1   
                             /teams/MIN/2023.html       1                   1   
                             /teams/NOP/2023.html       0                   0   
                             /teams/NYK/2023.html       0                   1   
                             /teams/OKC/2023.html       1                   1   
                             /teams/ORL/2023.html       0                   0   
                             /teams/PHI/2023.html       1                   1   
                             /teams/PHO/2023.html       0                   1   
                             /teams/POR/2023.html       0                   0   
                             /teams/SAC/2023.html       0                   0   
                             /teams/SAS/2023.html       1                   0   
                             /teams/TOR/2023.html       1                   0   
                             /teams/UTA/2023.html       0                   0   
                             /teams/WAS/2023.html       0                   1   

                                                   KNeighborsClassifier   
                       index Team_id                                      
/leagues/NBA_2015.html 8     /teams/BOS/2015.html                     1  \
                             /teams/BRK/2015.html                     0   
                             /teams/CHI/2015.html                     0   
                             /teams/CHO/2015.html                     1   
                             /teams/CLE/2015.html                     1   
                             /teams/DEN/2015.html                     0   
                             /teams/DET/2015.html                     0   
                             /teams/GSW/2015.html                     1   
                             /teams/HOU/2015.html                     1   
                             /teams/LAC/2015.html                     1   
                             /teams/MEM/2015.html                     1   
                             /teams/MIL/2015.html                     0   
                             /teams/MIN/2015.html                     0   
                             /teams/NOP/2015.html                     0   
                             /teams/NYK/2015.html                     0   
...                                                                 ...   
/leagues/NBA_2023.html 81  

In [8]:
ModelStack.score(y_preds_d1_val,score_names='classification')

,accuracy,f1,precision,recall
AdaBoostClassifier,0.651035,0.649797,0.652110,0.647499
KNeighborsClassifier,0.586773,0.605676,0.579182,0.634709
LogisticRegression,0.649475,0.648671,0.650162,0.647187
MLPClassifier,0.624831,0.621048,0.627374,0.614849
RandomForestClassifier,0.622387,0.621344,0.623066,0.619632
RidgeClassifier,0.650463,0.649716,0.651107,0.648331
SVC,0.645160,0.642014,0.647756,0.636373


In [41]:
X_avg = X.mean(axis=1)

In [45]:
y

Unnamed: 3_level_4
                       index Team_id                                 
/leagues/NBA_2015.html 8     /teams/BOS/2015.html                   0
                             /teams/BRK/2015.html                   0
                             /teams/CHI/2015.html                   1
                             /teams/CHO/2015.html                   1
                             /teams/CLE/2015.html                   0
                             /teams/DEN/2015.html                   0
                             /teams/DET/2015.html                   1
                             /teams/GSW/2015.html                   1
                             /teams/HOU/2015.html                   1
                             /teams/LAC/2015.html                   0
                             /teams/MEM/2015.html                   1
                             /teams/MIL/2015.html                   0
                             /teams/MIN/2015.html                   0
                             /teams/NOP/2015.html                   0
                             /teams/NYK/2015.html                   0
...                                                               ...
/leagues/NBA_2023.html 81    /teams/MIA/2023.html                   1
                             /teams/MIL/2023.html                   0
                             /teams/MIN/2023.html                   1
                             /teams/NOP/2023.html                   0
                             /teams/NYK/2023.html                   0
                             /teams/OKC/2023.html                   1
                             /teams/ORL/2023.html                   0
                             /teams/PHI/2023.html                   1
                             /teams/PHO/2023.html                   0
                             /teams/POR/2023.html                   0
                             /teams/SAC/2023.html                   0
                             /teams/SAS/2023.html                   1
                             /teams/TOR/2023.html                   1
                             /teams/UTA/2023.html                   0
                             /teams/WAS/2023.html                   0

[19234 rows x 1 columns]

In [56]:
# LOSS_THRESHOLD = 0.25
WIN_THRESHOLD = 0.33
# LOSS_PRED_IDX = X_avg[X_avg <= LOSS_THRESHOLD].index
# WIN_PRED_IDX = X_avg[X_avg >= WIN_THRESHOLD].index
# MIXED_PRED_IDX = X_avg[(X_avg > LOSS_THRESHOLD) & (X_avg < WIN_THRESHOLD)].index
# print(f1_score(y.loc[LOSS_PRED_IDX],X_avg.round().loc[LOSS_PRED_IDX]))
# print(f1_score(y.loc[WIN_PRED_IDX],X_avg.round().loc[WIN_PRED_IDX]))
# print(f1_score(y.loc[MIXED_PRED_IDX],X_avg.round().loc[MIXED_PRED_IDX]))
for WIN_THRESHOLD in np.linspace(0,1,21):
    WIN_PRED_IDX = X_avg[X_avg >= WIN_THRESHOLD].index
    X_avg_threshold = X_avg.copy()[WIN_PRED_IDX]
    X_avg_threshold[X_avg_threshold >= WIN_THRESHOLD] = 1
    print(WIN_THRESHOLD,len(WIN_PRED_IDX),f1_score(y.loc[WIN_PRED_IDX],X_avg_threshold))


0.0 19234 0.6666666666666666
0.05 15036 0.721537284244537
0.1 13741 0.7370404411764706
0.15000000000000002 12860 0.7472602406117578
0.2 12245 0.754323499491353
0.25 11703 0.7613251481795089
0.30000000000000004 11230 0.7667471996485833
0.35000000000000003 10806 0.7709281164695178
0.4 10430 0.7739508639943576
0.45 10048 0.7781358302425974
0.5 9693 0.7816742081447964
0.55 9171 0.7878667724028549
0.6000000000000001 8805 0.79234672884378
0.65 8400 0.7968201675857624
0.7000000000000001 7977 0.8024320672571686
0.75 7514 0.809286110450836
0.8 6981 0.8140660834111951
0.8500000000000001 6339 0.818783192024597
0.9 5507 0.8287962569119524
0.9500000000000001 4261 0.8468200270635994
1.0 2322 0.8555938886150813
